

# 1. Formulação do problema

### Variáveis de Decisão:
- **ti**: Tempo de pouso do avião i (variável contínua);


- **xij**: Variável binária que vale 1 se o avião i pousar antes do avião j, 0 caso contrário;


- **ei**: Adiantamento do tempo ideal de pouso para o avião i (ei ≥ 0);


- **di**: Atraso do tempo ideal de pouso para o avião i (di ≥ 0).

### Parâmetros:
- **Ri**: Tempo de sua detecção pelo radar;


- **Ei**: Tempo inicial de pouso;


- **Ti**: Tempo ideal para o pouso;


- **Li**: Tempo final que o avião i pode pousar;


### Função Objetivo:
Minimizar a penalidade total de pousos fora do tempo ideal:

   ### **Minimizar ∑ (gi * ei + hi * di)**

Onde:
- **gi** e **hi** são as penalidades por pousar antes ou depois do tempo ideal, respectivamente.

### Restrições:

#### **Restrições de Separação:**
O intervalo entre os pousos deve ser suficiente para garantir segurança e garantir uma sequência válida de aterrissagens:

- **tj ≥ ti + sij − M(1 − xij), ∀i,j, i≠j**  

    Se **xij = 1**, então **tj ≥ ti + sij**, garantindo a separação mínima entre os pousos.  


    Se **xij = 0**, a restrição é relaxada devido ao termo **M(1 − xij)**.

- **ti ≥ tj + sji − Mxij, ∀i,j, i≠j**  
            Similar à restrição anterior, mas garantindo que a relação entre **ti** e **tj** seja consistente dependendo do valor de    **xij**.

#### **Restrições de Sequenciamento:**
Garante que para cada par de aviões **(i, j)**, um deve pousar antes do outro, evitando ciclos:

- **xij + xji = 1, ∀i,j, i≠j**  
        Isso assegura que se o avião **i** pousar antes de **j**, então **xij = 1** e **xji = 0**, caso contrário, **xij = 0** e **xji = 1**.

#### **Cálculo de Adiantamento e Atraso:**

- **ei ≥ Ti − ti ∀i**;


- **di ≥ ti − Ti ∀i**;


- **ei ≤ Ti − Ei ∀i**;


- **di ≤ Li − Ti ∀i**.


# 2. Resolvendo usando o solver CBC

In [1]:
!pip install pulp
!sudo apt-get install glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 40.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 19 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0

In [14]:
import time
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, PULP_CBC_CMD, value

def ler_dados_instancia(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        linha_inicial = arquivo.readline().strip().split()
        qtdeDeAvioes = int(linha_inicial[0])

        tempos = []
        separacoes = []
        penalidades = []

        for _ in range(qtdeDeAvioes):
            linha_tempos = arquivo.readline().strip().split()
            while len(linha_tempos) < 6:
                linha_tempos.extend(arquivo.readline().strip().split())
            ri, ei, ti, li, gi, hi = map(float, linha_tempos[:6])
            tempos.append((ri, ei, ti, li, gi, hi))
            penalidades.append((gi, hi))

            linha_separacao = []
            while len(linha_separacao) < qtdeDeAvioes:
                linha_separacao.extend(arquivo.readline().strip().split())
            separacoes.append(list(map(float, linha_separacao[:qtdeDeAvioes])))

        return qtdeDeAvioes, tempos, separacoes, penalidades


def resolver_modelo_glpk(qtdeDeAvioes, tempos, separacoes, penalidades):
    modelo = LpProblem("SequenciamentoPousos", LpMinimize)

    # Variáveis de decisão
    t = [LpVariable(f"t_{i}", lowBound=tempos[i][1], upBound=tempos[i][3], cat='Continuous') for i in range(qtdeDeAvioes)]
    e = [LpVariable(f"e_{i}", lowBound=0, cat='Continuous') for i in range(qtdeDeAvioes)]
    d = [LpVariable(f"d_{i}", lowBound=0, cat='Continuous') for i in range(qtdeDeAvioes)]
    x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) if i != j else None for j in range(qtdeDeAvioes)] for i in range(qtdeDeAvioes)]

    # Função objetivo
    modelo += lpSum(penalidades[i][0] * e[i] + penalidades[i][1] * d[i] for i in range(qtdeDeAvioes)), "PenalidadeTotal"

    # Constante grande M
    M = max(tempos[i][3] for i in range(qtdeDeAvioes)) + max(max(separacoes[i]) for i in range(qtdeDeAvioes))

    # Restrições de separação e sequenciamento
    for i in range(qtdeDeAvioes):
        for j in range(qtdeDeAvioes):
            if i != j:
                modelo += t[j] >= t[i] + separacoes[i][j] - M * (1 - x[i][j]), f"Separacao_{i}_{j}_A"
                modelo += t[i] >= t[j] + separacoes[j][i] - M * x[i][j], f"Separacao_{j}_{i}_B"
                modelo += x[i][j] + x[j][i] == 1, f"Sequencia_{i}_{j}"

    # Cálculo de adiantamento e atraso
    for i in range(qtdeDeAvioes):
        modelo += e[i] >= tempos[i][2] - t[i], f"Adiantamento_{i}"
        modelo += d[i] >= t[i] - tempos[i][2], f"Atraso_{i}"
        modelo += e[i] <= tempos[i][2] - tempos[i][1], f"LimiteAdiantamento_{i}"
        modelo += d[i] <= tempos[i][3] - tempos[i][2], f"LimiteAtraso_{i}"

    # Resolver o modelo com tempo limite
    inicio_tempo = time.time()
    resultado = modelo.solve(PULP_CBC_CMD(msg=True, timeLimit=640))
    tempo_execucao = time.time() - inicio_tempo

    if modelo.status == 1:  # Solução ótima encontrada
        tempos_pouso = [value(t[i]) for i in range(qtdeDeAvioes)]
        valor_otimo = value(modelo.objective)
        print(f"Valor ótimo da solução: {valor_otimo}")
        print(f"Tempos de pouso: {', '.join(map(str, tempos_pouso))}")
        print(f"Tempo de execução do Solver: {tempo_execucao:.2f} segundos")
    else:  # Solução aproximada
        tempos_pouso = [value(t[i]) for i in range(qtdeDeAvioes) if t[i].varValue is not None]
        valor_aproximado = value(modelo.objective) if value(modelo.objective) is not None else float('inf')
        print("Solução aproximada encontrada (não ótima dentro do tempo limite)")
        print(f"Penalidade aproximada: {valor_aproximado}")
        print(f"Tempos de pouso aproximados: {', '.join(map(str, tempos_pouso))}")
        print(f"Tempo de execução do Solver: {tempo_execucao:.2f} segundos")


def executarInstancia():
    for i in range(1, 9):
        caminho_arquivo = f"problema_do_aviao/instances/0{i}.dat"
        qtdeDeAvioes, tempos, separacoes, penalidades = ler_dados_instancia(caminho_arquivo)
        resolver_modelo_glpk(qtdeDeAvioes, tempos, separacoes, penalidades)


executarInstancia()

Valor ótimo da solução: 699.999999999892
Tempos de pouso: 165.0, 258.0, 98.0, 106.0, 118.0, 126.0, 134.0, 142.0, 150.0, 180.0
Tempo de execução do Solver: 0.25 segundos
Valor ótimo da solução: 1480.0
Tempos de pouso: 196.0, 250.0, 90.0, 98.0, 106.0, 130.0, 122.0, 114.0, 138.0, 151.0, 341.0, 313.0, 181.0, 171.0, 344.0
Tempo de execução do Solver: 2.39 segundos
Valor ótimo da solução: 820.0
Tempos de pouso: 82.0, 197.0, 184.0, 117.0, 261.0, 101.0, 229.0, 109.0, 133.0, 141.0, 149.0, 125.0, 336.0, 316.0, 258.0, 409.0, 339.0, 287.0, 160.0, 169.0
Tempo de execução do Solver: 0.91 segundos
Valor ótimo da solução: 2520.0
Tempos de pouso: 82.0, 90.0, 201.0, 270.0, 98.0, 122.0, 130.0, 114.0, 106.0, 280.0, 295.0, 146.0, 138.0, 291.0, 186.0, 154.0, 170.0, 178.0, 162.0, 357.0
Tempo de execução do Solver: 22.43 segundos
Valor ótimo da solução: 3100.0
Tempos de pouso: 201.0, 246.0, 82.0, 90.0, 98.0, 122.0, 114.0, 106.0, 130.0, 138.0, 307.0, 280.0, 170.0, 146.0, 301.0, 393.0, 162.0, 178.0, 154.0, 186.

# 3. Resolvendo usando a Heurística (VNS)

In [13]:
import random
import time

def ler_dados_instancia(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        linha_inicial = arquivo.readline().strip().split()
        qtdeDeAvioes = int(linha_inicial[0])

        tempos = []
        separacoes = []
        penalidades = []

        for _ in range(qtdeDeAvioes):
            linha_tempos = arquivo.readline().strip().split()
            while len(linha_tempos) < 6:
                linha_tempos.extend(arquivo.readline().strip().split())
            ri, ei, ti, li, gi, hi = map(float, linha_tempos[:6])
            tempos.append((ri, ei, ti, li, gi, hi))
            penalidades.append((gi, hi))

            linha_separacao = []
            while len(linha_separacao) < qtdeDeAvioes:
                linha_separacao.extend(arquivo.readline().strip().split())
            separacoes.append(list(map(float, linha_separacao[:qtdeDeAvioes])))

        return qtdeDeAvioes, tempos, separacoes, penalidades

def calcular_penalidade(sequencia, tempos, separacoes, penalidades):
    n = len(sequencia)
    tempos_pouso = [0] * n
    penalidade_total = 0

    for index, i in enumerate(sequencia):
        if index == 0:
            tempos_pouso[i] = tempos[i][1]  # Começa no tempo inicial permitido
        else:
            prev = sequencia[index - 1]
            tempos_pouso[i] = max(tempos[i][1], tempos_pouso[prev] + separacoes[prev][i])

        # Calcula a penalidade
        atraso = max(0, tempos_pouso[i] - tempos[i][2])
        adiantamento = max(0, tempos[i][2] - tempos_pouso[i])
        penalidade_total += atraso * penalidades[i][1] + adiantamento * penalidades[i][0]

    return penalidade_total, tempos_pouso

def gerar_solucao_inicial(n, tempos, penalidades):
    # Solução inicial ordenada com base no tempo ideal (tempos[i][2])
    indices_ordenados = sorted(range(n), key=lambda i: tempos[i][2])
    return indices_ordenados  # Retorna a sequência ordenada com base no tempo ideal

def perturbar_solucao(solucao):
    # Perturbação simples: troca de posição entre dois aviões
    i, j = random.sample(range(len(solucao)), 2)
    nova_solucao = solucao[:]
    nova_solucao[i], nova_solucao[j] = nova_solucao[j], nova_solucao[i]
    return nova_solucao

def perturbar_solucao_blocos(solucao, bloco_tamanho=5):
    # Perturbação de troca de blocos de aviões
    n = len(solucao)
    i = random.randint(0, n - bloco_tamanho)  # Posição inicial do bloco
    j = random.randint(0, n - bloco_tamanho)  # Posição final do bloco

    nova_solucao = solucao[:]
    nova_solucao[i:i+bloco_tamanho], nova_solucao[j:j+bloco_tamanho] = nova_solucao[j:j+bloco_tamanho], nova_solucao[i:i+bloco_tamanho]
    return nova_solucao

def vns_perturbacao_2(solucao):
    # Outra perturbação: deslocamento circular de um avião
    nova_solucao = solucao[:]
    avião = nova_solucao.pop(random.randint(0, len(nova_solucao) - 1))
    nova_solucao.append(avião)
    return nova_solucao

def variable_neighbourhood_search(qtdeDeAvioes, tempos, separacoes, penalidades, max_iter=5000):
    melhor_solucao = gerar_solucao_inicial(qtdeDeAvioes, tempos, penalidades)
    melhor_penalidade, _ = calcular_penalidade(melhor_solucao, tempos, separacoes, penalidades)

    # Print da solução inicial e sua penalidade
    print(f"Solução inicial: {melhor_solucao}")
    print(f"Penalidade da solução inicial: {melhor_penalidade}")

    iteracao = 0
    while iteracao < max_iter:
        # Alterna entre diferentes tipos de perturbações
        if iteracao % 3 == 0:
            nova_solucao = perturbar_solucao(melhor_solucao)
        elif iteracao % 3 == 1:
            nova_solucao = perturbar_solucao_blocos(melhor_solucao, bloco_tamanho=5)  # Troca blocos de tamanho 5
        else:
            nova_solucao = vns_perturbacao_2(melhor_solucao)

        nova_penalidade, _ = calcular_penalidade(nova_solucao, tempos, separacoes, penalidades)

        if nova_penalidade < melhor_penalidade:
            melhor_solucao, melhor_penalidade = nova_solucao, nova_penalidade
            iteracao = 0  # Reinicia o contador de iteração após encontrar uma solução melhor
        else:
            iteracao += 1

    return melhor_solucao, melhor_penalidade

def executar_vns(caminho_arquivo, inst_num):
    qtdeDeAvioes, tempos, separacoes, penalidades = ler_dados_instancia(caminho_arquivo)
    inicio_tempo = time.time()
    melhor_sequencia, melhor_penalidade = variable_neighbourhood_search(qtdeDeAvioes, tempos, separacoes, penalidades)
    tempo_execucao = time.time() - inicio_tempo

    print(f"Resultados da Instância {inst_num}:")
    print(f"Melhor penalidade encontrada: {melhor_penalidade}")
    print(f"Melhor sequência de pousos: {melhor_sequencia}")
    print(f"Tempo de execução: {tempo_execucao:.2f} segundos")
    print("============================================")

for i in range(1, 9):
    caminho_arquivo = f"problema_do_aviao/instances/0{i}.dat"
    executar_vns(caminho_arquivo, i)

Solução inicial: [2, 3, 4, 5, 6, 7, 8, 0, 9, 1]
Penalidade da solução inicial: 2830.0
Resultados da Instância 1:
Melhor penalidade encontrada: 2110.0
Melhor sequência de pousos: [3, 2, 5, 4, 6, 7, 8, 0, 9, 1]
Tempo de execução: 0.05 segundos
Solução inicial: [2, 3, 4, 5, 7, 6, 8, 9, 0, 13, 12, 1, 11, 10, 14]
Penalidade da solução inicial: 4520.0
Resultados da Instância 2:
Melhor penalidade encontrada: 3380.0
Melhor sequência de pousos: [2, 3, 2, 4, 5, 7, 8, 9, 0, 13, 12, 14, 1, 11, 10]
Tempo de execução: 0.08 segundos
Solução inicial: [0, 5, 7, 3, 11, 9, 8, 10, 2, 18, 19, 1, 6, 14, 4, 17, 13, 12, 16, 15]
Penalidade da solução inicial: 6990.0
Resultados da Instância 3:
Melhor penalidade encontrada: 2490.0
Melhor sequência de pousos: [0, 7, 5, 3, 11, 17, 6, 14, 4, 14, 4, 14, 4, 14, 4, 14, 4, 14, 4, 17]
Tempo de execução: 0.15 segundos
Solução inicial: [0, 1, 4, 8, 7, 5, 6, 12, 11, 15, 18, 17, 16, 14, 2, 3, 9, 13, 10, 19]
Penalidade da solução inicial: 5610.0
Resultados da Instância 4:
Me